In [1]:
import pandas as pd
import numpy as np
import nltk
import pyLDAvis.sklearn
from collections import Counter
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.options.mode.chained_assignment = None
pd.set_option('display.max_colwidth', 100)

c:\Users\cheng\Anaconda3\lib\site-packages\matplotlib\__init__.py:169: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(module.__version__) < minver:
c:\Users\cheng\Anaconda3\lib\site-packages\setuptools\_distutils\version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
c:\Users\cheng\Anaconda3\lib\site-packages\matplotlib\__init__.py:169: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(module.__version__) < minver:
c:\Users\cheng\Anaconda3\lib\site-packages\setuptools\_distutils\version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
c:\Users\cheng\Anaconda3\lib\site-packages\matplotlib\__init__.py:169: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if

In [33]:
# import data
df = pd.read_csv('BCS_processed_audience_reviews.csv')[['rating_stars', 'season', 'review_text_processed']]

In [34]:
# sentiment analysis
df['review_text_processed'] = [' '.join(map(str, word)) for word in df['review_text_processed']]
df['sentiment'] = df['review_text_processed'].apply(lambda x: TextBlob(x).sentiment.polarity)

In [31]:
plt.figure(figsize=(15, 8))
plt.margins(0.02)
plt.xlabel('Sentiment', fontsize=18)
plt.xticks(fontsize=18)
plt.ylabel('Frequency', fontsize=18)
plt.yticks(fontsize=18)
plt.hist(df['sentiment'], bins=50)
plt.title('Sentiment Distribution', fontsize=18)
plt.show()

array([0.])

In [35]:
df.head(20)

,rating_stars,season,review_text_processed,sentiment
0,3.5,1,"[ ' w e l l ' , ' c a l l ' , ' s a u l ' , ' s e a s o n ' , ' 1 ' , ' g o o d ' , ...",0.0
1,5.0,1,"[ ' g o o d ' , ' g o o d ' , ' p r e q u e l ' , ' a l l a h ' , ' g r a n t ' , ' u ...",0.0
2,2.0,1,"[ ' l i k e ' , ' b r e a k ' , ' b a d ' , ' e x c e p t ' , ' e x t r e m e l y ' , ...",0.0
3,5.0,1,"[ ' a b s o l u t e ' , ' b r i l l i a n t ' , ' s e a s o n ' , ' s t a r t ' , ' s h ...",0.0
4,4.5,1,"[ ' i n c r e d i b l e ' , ' w r i t e ' , ' r e a l i s t i c ' , ' p e r f o r m a n c ...",0.0
5,5.0,1,"[ ' j i m m y ' , ' m c d o n a l d s ' , ' e l e c t r o c u t e ' , ' b r o t h e r ' , ...",0.0
6,5.0,1,"[ ' o h ' , ' s h o w ' , ' e v e n ' , ' g r e a t ' , ' b r e a k ' , ' b a d ' , ...",0.0
7,5.0,1,"[ ' s l i g h t l y ' , ' d i f f e r e n t ' , ' f e e l ' , ' ` ` ' , ' b r e a k ' , ...",0.0
8,5.0,1,"[ ' a b s o l u t e l y ' , ' b r i l l i a n t ' , ' w a n t ' , ' k n o w ' , ' p e r ...",0.0
9,4.5,1,"[ ' i n c r e d i b l y ' , ' s u r p r i s i n g ' , ' p r e q u e l ' , ' p r e v i o u ...",0.0
